In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [1]:
import gc
import numpy as np
import pandas as pd
import tensorflow as tf
import xgboost as xgb

from collections import defaultdict
from sklearn.preprocessing import OneHotEncoder, label_binarize
from sklearn.metrics import accuracy_score, classification_report, balanced_accuracy_score, roc_auc_score, average_precision_score, fbeta_score, matthews_corrcoef
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization, Input, Conv1D, Lambda, Flatten

In [2]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [3]:
def create_convnext_like_feature_extractor(input_dim, sliding_window_size=3):
    input_layer = Input(shape=(input_dim,))
    reshaped_input = Lambda(lambda x: tf.expand_dims(x, axis=-1))(input_layer)
    x = reshaped_input

    # Simulated ConvNeXt block
    x = Conv1D(filters=int(max(16, input_dim * 2)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    x = Conv1D(filters=int(max(16, input_dim)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    x = Conv1D(filters=int(max(16, input_dim // 2)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    x = Conv1D(filters=int(max(16, input_dim // 4)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    x = Conv1D(filters=int(max(16, input_dim // 8)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)

    # Flatten the output for feature extraction
    feature_output = Flatten()(x)

    # Reconstruction head for self-supervised learning
    reconstruction_output = Dense(input_dim, activation='linear')(feature_output)

    # Create the model
    model = Model(inputs=input_layer, outputs=reconstruction_output)
    return model

In [4]:
def focal_loss_eval(predt, dtrain, gamma=2.0):
    """ Custom evaluation metric based on focal loss for multi-class classification. """
    labels = dtrain.get_label().astype(int)
    predt = predt.reshape(len(labels), -1)  # Reshape to (num_samples, num_classes)
    
    # Convert to softmax probabilities
    exp_preds = np.exp(predt - np.max(predt, axis=1, keepdims=True))
    probs = exp_preds / np.sum(exp_preds, axis=1, keepdims=True)
    
    # Get the probability of the true class
    pt = probs[np.arange(len(labels)), labels]
    
    # Compute the focal loss (evaluation)
    focal_loss = -((1 - pt) ** gamma) * np.log(pt)
    
    return 'focal_loss', np.mean(focal_loss)

In [5]:
def run_model(train_data_path, test_data_path, is_string_labels = False, label_mapping = None):

    # Load and Prepare Training Data
    train_data = import_data(train_data_path)
    train_data = train_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    train_X = train_data.drop(['label'], axis=1)
    if (is_string_labels):
        train_y = train_data['label'].map(label_mapping)
    else:
        train_y = train_data['label'] - 1
    del train_data
    gc.collect()

    # Load and Prepare Test Data (this will not be used in training)
    test_data = import_data(test_data_path)
    test_data = test_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    test_X = test_data.drop(['label'], axis=1)
    if (is_string_labels):
        test_y = test_data['label'].map(label_mapping)
    else:
        test_y = test_data['label'] - 1
    test_y_binarized = label_binarize(test_y, classes=range(len(np.unique(test_y))))
    del test_data
    gc.collect()

    # Number of runs for averaging results
    num_runs = 50

    # Initialize storage for metrics
    metrics_storage = defaultdict(list)

    # Define XGBoost parameters
    params = {
        "num_class": len(train_y.unique()),
        "disable_default_eval_metric": True,
        "max_depth": 6,
        "eta": 0.3,
        "seed": 42
    }

    # Train the Model with Validation Split N tines for more accurate metrics
    #print("Verbose output only for first run...")
    verbose_run = 0
    for run in range(num_runs):
        print(f"Run {run + 1}/{num_runs} started...")
                
        # Train the NN feature extractor
        feature_extractor = create_convnext_like_feature_extractor(train_X.shape[1])
        feature_extractor.compile(optimizer='adam', loss='categorical_focal_crossentropy', metrics=['accuracy'])
        feature_extractor.fit(train_X, train_X, epochs=3, batch_size=32, verbose=0)  # Self-supervised training

        # Extract features
        feature_extractor_truncated = Model(inputs=feature_extractor.input, outputs=feature_extractor.layers[-2].output)
        train_features = feature_extractor_truncated.predict(train_X, verbose=0)
        test_features = feature_extractor_truncated.predict(test_X, verbose=0)
        del feature_extractor
        gc.collect()

        train_combined = np.hstack((train_X, train_features))
        test_combined = np.hstack((test_X, test_features))
        del train_features
        del test_features
        gc.collect()

        # Convert data to DMatrix
        dtrain = xgb.DMatrix(data=train_combined, label=train_y)
        dtest = xgb.DMatrix(data=test_combined, label=test_y)
        del train_combined
        del test_combined
        gc.collect()

        # Train XGBoost
        num_round = 100
        bst = xgb.train(params, dtrain, num_round, custom_metric=focal_loss_eval)

        # Make predictions
        test_predictions = bst.predict(dtest)
        del dtrain
        del dtest
        gc.collect()

        # Compute metrics
        accuracy = accuracy_score(test_y, test_predictions)
        balanced_acc = balanced_accuracy_score(test_y, test_predictions)
        f2 = fbeta_score(test_y, test_predictions, beta=2, average='weighted')
        mcc = matthews_corrcoef(test_y, test_predictions)

        # Store metrics
        metrics_storage['test_accuracy'].append(accuracy)
        metrics_storage['balanced_accuracy'].append(balanced_acc)
        metrics_storage['f2'].append(f2)
        metrics_storage['mcc'].append(mcc)

        # Store classification report metrics
        report = classification_report(test_y, test_predictions, output_dict=True)
        for label, values in report.items():
            # Check if the value is a dictionary (e.g., 'precision', 'recall', 'f1-score')
            if isinstance(values, dict):
                for metric, value in values.items():
                    metrics_storage[f"{label}_{metric}"].append(value)
            else:
                # Handle scalar values (like 'accuracy')
                metrics_storage[label].append(values)

        # Average the metrics over all successful runs
        print(f"\nAggregated Metrics for {run+1} runs:")
        for metric, values in metrics_storage.items():
            avg_value = np.mean(values)
            print(f"{metric}: {avg_value:.4f}")

    gc.collect()

In [ ]:
run_model("shuttle_bsmote.csv", "shuttle_test.csv", is_string_labels = False)

Run 1/50 started...


Aggregated Metrics for 1 runs:
test_accuracy: 0.9994
balanced_accuracy: 0.9281
f2: 0.9994
mcc: 0.9983
0_precision: 1.0000
0_recall: 0.9997
0_f1-score: 0.9998
0_support: 9117.0000
1_precision: 0.9091
1_recall: 1.0000
1_f1-score: 0.9524
1_support: 10.0000
2_precision: 0.8718
2_recall: 1.0000
2_f1-score: 0.9315
2_support: 34.0000
3_precision: 1.0000
3_recall: 0.9989
3_f1-score: 0.9994
3_support: 1781.0000
4_precision: 0.9985
4_recall: 0.9985
4_f1-score: 0.9985
4_support: 653.0000
5_precision: 1.0000
5_recall: 0.5000
5_f1-score: 0.6667
5_support: 2.0000
6_precision: 1.0000
6_recall: 1.0000
6_f1-score: 1.0000
6_support: 3.0000
accuracy: 0.9994
macro avg_precision: 0.9685
macro avg_recall: 0.9281
macro avg_f1-score: 0.9355
macro avg_support: 11600.0000
weighted avg_precision: 0.9995
weighted avg_recall: 0.9994
weighted avg_f1-score: 0.9994
weighted avg_support: 11600.0000
Run 2/50 started...

Aggregated Metrics for 2 runs:
test_accuracy: 0.9993
balanced_accuracy: 0.9281

d:\Uni Things\Masterarbeit\NNs\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Uni Things\Masterarbeit\NNs\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Uni Things\Masterarbeit\NNs\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Aggregated Metrics for 19 runs:
test_accuracy: 0.9994
balanced_accuracy: 0.9498
f2: 0.9994
mcc: 0.9982
0_precision: 1.0000
0_recall: 0.9996
0_f1-score: 0.9998
0_support: 9117.0000
1_precision: 0.9091
1_recall: 1.0000
1_f1-score: 0.9524
1_support: 10.0000
2_precision: 0.8633
2_recall: 0.9938
2_f1-score: 0.9238
2_support: 34.0000
3_precision: 0.9999
3_recall: 0.9988
3_f1-score: 0.9994
3_support: 1781.0000
4_precision: 0.9990
4_recall: 0.9986
4_f1-score: 0.9988
4_support: 653.0000
5_precision: 0.9474
5_recall: 0.6579
5_f1-score: 0.7544
5_support: 2.0000
6_precision: 0.9737
6_recall: 1.0000
6_f1-score: 0.9850
6_support: 3.0000
accuracy: 0.9994
macro avg_precision: 0.9560
macro avg_recall: 0.9498
macro avg_f1-score: 0.9448
macro avg_support: 11600.0000
weighted avg_precision: 0.9994
weighted avg_recall: 0.9994
weighted avg_f1-score: 0.9994
weighted avg_support: 11600.0000
Run 20/50 started...

Aggregated Metrics for 20 runs:
test_accuracy: 0.9994
balanced_accuracy: 0.9487
f2: 0.9994
mcc: 0

d:\Uni Things\Masterarbeit\NNs\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Uni Things\Masterarbeit\NNs\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Uni Things\Masterarbeit\NNs\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Aggregated Metrics for 22 runs:
test_accuracy: 0.9993
balanced_accuracy: 0.9434
f2: 0.9993
mcc: 0.9982
0_precision: 1.0000
0_recall: 0.9996
0_f1-score: 0.9998
0_support: 9117.0000
1_precision: 0.9056
1_recall: 1.0000
1_f1-score: 0.9504
1_support: 10.0000
2_precision: 0.8633
2_recall: 0.9933
2_f1-score: 0.9236
2_support: 34.0000
3_precision: 0.9998
3_recall: 0.9989
3_f1-score: 0.9993
3_support: 1781.0000
4_precision: 0.9989
4_recall: 0.9986
4_f1-score: 0.9987
4_support: 653.0000
5_precision: 0.9091
5_recall: 0.6136
5_f1-score: 0.7121
5_support: 2.0000
6_precision: 0.9773
6_recall: 1.0000
6_f1-score: 0.9870
6_support: 3.0000
accuracy: 0.9993
macro avg_precision: 0.9506
macro avg_recall: 0.9434
macro avg_f1-score: 0.9387
macro avg_support: 11600.0000
weighted avg_precision: 0.9994
weighted avg_recall: 0.9993
weighted avg_f1-score: 0.9994
weighted avg_support: 11600.0000
Run 23/50 started...

Aggregated Metrics for 23 runs:
test_accuracy: 0.9994
balanced_accuracy: 0.9428
f2: 0.9993
mcc: 0

d:\Uni Things\Masterarbeit\NNs\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Uni Things\Masterarbeit\NNs\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Uni Things\Masterarbeit\NNs\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Aggregated Metrics for 32 runs:
test_accuracy: 0.9994
balanced_accuracy: 0.9452
f2: 0.9994
mcc: 0.9982
0_precision: 1.0000
0_recall: 0.9996
0_f1-score: 0.9998
0_support: 9117.0000
1_precision: 0.9044
1_recall: 1.0000
1_f1-score: 0.9497
1_support: 10.0000
2_precision: 0.8674
2_recall: 0.9945
2_f1-score: 0.9264
2_support: 34.0000
3_precision: 0.9998
3_recall: 0.9989
3_f1-score: 0.9994
3_support: 1781.0000
4_precision: 0.9989
4_recall: 0.9985
4_f1-score: 0.9987
4_support: 653.0000
5_precision: 0.9062
5_recall: 0.6250
5_f1-score: 0.7188
5_support: 2.0000
6_precision: 0.9844
6_recall: 1.0000
6_f1-score: 0.9911
6_support: 3.0000
accuracy: 0.9994
macro avg_precision: 0.9516
macro avg_recall: 0.9452
macro avg_f1-score: 0.9405
macro avg_support: 11600.0000
weighted avg_precision: 0.9994
weighted avg_recall: 0.9994
weighted avg_f1-score: 0.9994
weighted avg_support: 11600.0000
Run 33/50 started...

Aggregated Metrics for 33 runs:
test_accuracy: 0.9994
balanced_accuracy: 0.9467
f2: 0.9994
mcc: 0

In [ ]:
run_model("/kaggle/input/ma-bsmote/covtype_bsmote.csv", "/kaggle/input/ma-datasets/covtype_test.csv", is_string_labels = False)

Run 1/50 started...



Aggregated Metrics for 1 runs:
test_accuracy: 0.8776
balanced_accuracy: 0.8786
f2: 0.8775
mcc: 0.8040
0_precision: 0.8854
0_recall: 0.8412
0_f1-score: 0.8628
0_support: 42368.0000
1_precision: 0.8794
1_recall: 0.9037
1_f1-score: 0.8914
1_support: 56661.0000
2_precision: 0.9232
2_recall: 0.8644
2_f1-score: 0.8928
2_support: 7151.0000
3_precision: 0.7262
3_recall: 0.9180
3_f1-score: 0.8109
3_support: 549.0000
4_precision: 0.6899
4_recall: 0.7967
4_f1-score: 0.7395
4_support: 1899.0000
5_precision: 0.7832
5_recall: 0.8739
5_f1-score: 0.8261
5_support: 3473.0000
6_precision: 0.9160
6_recall: 0.9522
6_f1-score: 0.9338
6_support: 4102.0000
accuracy: 0.8776
macro avg_precision: 0.8291
macro avg_recall: 0.8786
macro avg_f1-score: 0.8510
macro avg_support: 116203.0000
weighted avg_precision: 0.8789
weighted avg_recall: 0.8776
weighted avg_f1-score: 0.8777
weighted avg_support: 116203.0000
Run 2/50 started...



Aggregated Metrics for 2 runs:
test_accuracy: 0.8777
balanced_accuracy: 0.8796
f2: 0.8776
mcc: 0.8041
0_precision: 0.8845
0_recall: 0.8419
0_f1-score: 0.8627
0_support: 42368.0000
1_precision: 0.8793
1_recall: 0.9027
1_f1-score: 0.8909
1_support: 56661.0000
2_precision: 0.9246
2_recall: 0.8687
2_f1-score: 0.8958
2_support: 7151.0000
3_precision: 0.7304
3_recall: 0.9226
3_f1-score: 0.8153
3_support: 549.0000
4_precision: 0.6914
4_recall: 0.7938
4_f1-score: 0.7391
4_support: 1899.0000
5_precision: 0.7907
5_recall: 0.8755
5_f1-score: 0.8309
5_support: 3473.0000
6_precision: 0.9151
6_recall: 0.9522
6_f1-score: 0.9333
6_support: 4102.0000
accuracy: 0.8777
macro avg_precision: 0.8308
macro avg_recall: 0.8796
macro avg_f1-score: 0.8525
macro avg_support: 116203.0000
weighted avg_precision: 0.8788
weighted avg_recall: 0.8777
weighted avg_f1-score: 0.8777
weighted avg_support: 116203.0000
Run 3/50 started...



Aggregated Metrics for 3 runs:
test_accuracy: 0.8787
balanced_accuracy: 0.8806
f2: 0.8786
mcc: 0.8057
0_precision: 0.8861
0_recall: 0.8426
0_f1-score: 0.8638
0_support: 42368.0000
1_precision: 0.8798
1_recall: 0.9040
1_f1-score: 0.8918
1_support: 56661.0000
2_precision: 0.9259
2_recall: 0.8692
2_f1-score: 0.8966
2_support: 7151.0000
3_precision: 0.7333
3_recall: 0.9229
3_f1-score: 0.8172
3_support: 549.0000
4_precision: 0.6928
4_recall: 0.7953
4_f1-score: 0.7405
4_support: 1899.0000
5_precision: 0.7917
5_recall: 0.8759
5_f1-score: 0.8317
5_support: 3473.0000
6_precision: 0.9161
6_recall: 0.9542
6_f1-score: 0.9348
6_support: 4102.0000
accuracy: 0.8787
macro avg_precision: 0.8322
macro avg_recall: 0.8806
macro avg_f1-score: 0.8538
macro avg_support: 116203.0000
weighted avg_precision: 0.8798
weighted avg_recall: 0.8787
weighted avg_f1-score: 0.8788
weighted avg_support: 116203.0000
Run 4/50 started...



Aggregated Metrics for 4 runs:
test_accuracy: 0.8792
balanced_accuracy: 0.8820
f2: 0.8791
mcc: 0.8064
0_precision: 0.8862
0_recall: 0.8431
0_f1-score: 0.8641
0_support: 42368.0000
1_precision: 0.8805
1_recall: 0.9042
1_f1-score: 0.8922
1_support: 56661.0000
2_precision: 0.9267
2_recall: 0.8707
2_f1-score: 0.8978
2_support: 7151.0000
3_precision: 0.7296
3_recall: 0.9271
3_f1-score: 0.8165
3_support: 549.0000
4_precision: 0.6928
4_recall: 0.7991
4_f1-score: 0.7422
4_support: 1899.0000
5_precision: 0.7949
5_recall: 0.8758
5_f1-score: 0.8334
5_support: 3473.0000
6_precision: 0.9158
6_recall: 0.9541
6_f1-score: 0.9346
6_support: 4102.0000
accuracy: 0.8792
macro avg_precision: 0.8324
macro avg_recall: 0.8820
macro avg_f1-score: 0.8544
macro avg_support: 116203.0000
weighted avg_precision: 0.8803
weighted avg_recall: 0.8792
weighted avg_f1-score: 0.8792
weighted avg_support: 116203.0000
Run 5/50 started...


...


Aggregated Metrics for 29 runs:
test_accuracy: 0.8800
balanced_accuracy: 0.8846
f2: 0.8799
mcc: 0.8078
0_precision: 0.8857
0_recall: 0.8450
0_f1-score: 0.8649
0_support: 42368.0000
1_precision: 0.8821
1_recall: 0.9039
1_f1-score: 0.8929
1_support: 56661.0000
2_precision: 0.9284
2_recall: 0.8717
2_f1-score: 0.8991
2_support: 7151.0000
3_precision: 0.7364
3_recall: 0.9327
3_f1-score: 0.8230
3_support: 549.0000
4_precision: 0.6958
4_recall: 0.8049
4_f1-score: 0.7463
4_support: 1899.0000
5_precision: 0.7960
5_recall: 0.8823
5_f1-score: 0.8369
5_support: 3473.0000
6_precision: 0.9153
6_recall: 0.9517
6_f1-score: 0.9331
6_support: 4102.0000
accuracy: 0.8800
macro avg_precision: 0.8343
macro avg_recall: 0.8846
macro avg_f1-score: 0.8566
macro avg_support: 116203.0000
weighted avg_precision: 0.8812
weighted avg_recall: 0.8800
weighted avg_f1-score: 0.8801
weighted avg_support: 116203.0000
Run 30/50 started...



Aggregated Metrics for 30 runs:
test_accuracy: 0.8800
balanced_accuracy: 0.8846
f2: 0.8799
mcc: 0.8078
0_precision: 0.8856
0_recall: 0.8449
0_f1-score: 0.8648
0_support: 42368.0000
1_precision: 0.8821
1_recall: 0.9039
1_f1-score: 0.8929
1_support: 56661.0000
2_precision: 0.9286
2_recall: 0.8719
2_f1-score: 0.8993
2_support: 7151.0000
3_precision: 0.7362
3_recall: 0.9327
3_f1-score: 0.8229
3_support: 549.0000
4_precision: 0.6960
4_recall: 0.8048
4_f1-score: 0.7464
4_support: 1899.0000
5_precision: 0.7966
5_recall: 0.8826
5_f1-score: 0.8374
5_support: 3473.0000
6_precision: 0.9153
6_recall: 0.9516
6_f1-score: 0.9331
6_support: 4102.0000
accuracy: 0.8800
macro avg_precision: 0.8343
macro avg_recall: 0.8846
macro avg_f1-score: 0.8567
macro avg_support: 116203.0000
weighted avg_precision: 0.8811
weighted avg_recall: 0.8800
weighted avg_f1-score: 0.8801
weighted avg_support: 116203.0000
Run 31/50 started...



Aggregated Metrics for 31 runs:
test_accuracy: 0.8799
balanced_accuracy: 0.8844
f2: 0.8799
mcc: 0.8077
0_precision: 0.8856
0_recall: 0.8449
0_f1-score: 0.8648
0_support: 42368.0000
1_precision: 0.8820
1_recall: 0.9039
1_f1-score: 0.8928
1_support: 56661.0000
2_precision: 0.9285
2_recall: 0.8717
2_f1-score: 0.8992
2_support: 7151.0000
3_precision: 0.7361
3_recall: 0.9324
3_f1-score: 0.8226
3_support: 549.0000
4_precision: 0.6961
4_recall: 0.8046
4_f1-score: 0.7464
4_support: 1899.0000
5_precision: 0.7960
5_recall: 0.8822
5_f1-score: 0.8369
5_support: 3473.0000
6_precision: 0.9152
6_recall: 0.9515
6_f1-score: 0.9330
6_support: 4102.0000
accuracy: 0.8799
macro avg_precision: 0.8342
macro avg_recall: 0.8844
macro avg_f1-score: 0.8565
macro avg_support: 116203.0000
weighted avg_precision: 0.8811
weighted avg_recall: 0.8799
weighted avg_f1-score: 0.8800
weighted avg_support: 116203.0000
Run 32/50 started...



Aggregated Metrics for 32 runs:
test_accuracy: 0.8798
balanced_accuracy: 0.8844
f2: 0.8798
mcc: 0.8075
0_precision: 0.8854
0_recall: 0.8448
0_f1-score: 0.8646
0_support: 42368.0000
1_precision: 0.8820
1_recall: 0.9037
1_f1-score: 0.8927
1_support: 56661.0000
2_precision: 0.9285
2_recall: 0.8717
2_f1-score: 0.8992
2_support: 7151.0000
3_precision: 0.7366
3_recall: 0.9323
3_f1-score: 0.8229
3_support: 549.0000
4_precision: 0.6961
4_recall: 0.8046
4_f1-score: 0.7464
4_support: 1899.0000
5_precision: 0.7959
5_recall: 0.8824
5_f1-score: 0.8369
5_support: 3473.0000
6_precision: 0.9152
6_recall: 0.9514
6_f1-score: 0.9330
6_support: 4102.0000
accuracy: 0.8798
macro avg_precision: 0.8342
macro avg_recall: 0.8844
macro avg_f1-score: 0.8565
macro avg_support: 116203.0000
weighted avg_precision: 0.8810
weighted avg_recall: 0.8798
weighted avg_f1-score: 0.8799
weighted avg_support: 116203.0000
Run 33/50 started...



Aggregated Metrics for 33 runs:
test_accuracy: 0.8798
balanced_accuracy: 0.8844
f2: 0.8797
mcc: 0.8074
0_precision: 0.8853
0_recall: 0.8447
0_f1-score: 0.8645
0_support: 42368.0000
1_precision: 0.8819
1_recall: 0.9037
1_f1-score: 0.8927
1_support: 56661.0000
2_precision: 0.9284
2_recall: 0.8717
2_f1-score: 0.8992
2_support: 7151.0000
3_precision: 0.7368
3_recall: 0.9323
3_f1-score: 0.8230
3_support: 549.0000
4_precision: 0.6964
4_recall: 0.8049
4_f1-score: 0.7467
4_support: 1899.0000
5_precision: 0.7960
5_recall: 0.8823
5_f1-score: 0.8369
5_support: 3473.0000
6_precision: 0.9152
6_recall: 0.9512
6_f1-score: 0.9328
6_support: 4102.0000
accuracy: 0.8798
macro avg_precision: 0.8343
macro avg_recall: 0.8844
macro avg_f1-score: 0.8566
macro avg_support: 116203.0000
weighted avg_precision: 0.8809
weighted avg_recall: 0.8798
weighted avg_f1-score: 0.8798
weighted avg_support: 116203.0000
Run 34/50 started...


In [ ]:
labels_map = {
    'normal.': 0, 'satan.': 1, 'ipsweep.': 2, 'portsweep.': 3, 'nmap.': 4,
    'back.': 5, 'warezclient.': 6, 'teardrop.': 7, 'pod.': 8, 'guess_passwd.': 9,
    'buffer_overflow.': 10, 'land.': 11, 'warezmaster.': 12, 'imap.': 13, 'rootkit.': 14,
    'loadmodule.': 15, 'multihop.': 16, 'ftp_write.': 17, 'phf.': 18, 'perl.': 19, 'spy.': 20
}

run_model("kdd_bsmote.csv", "kdd_test.csv", is_string_labels = True, label_mapping=labels_map)

C:\Users\malyarchuk\AppData\Local\Temp\ipykernel_24244\147790395.py:42: FutureWarning: The 'keep_date_col' keyword in pd.read_csv is deprecated and will be removed in a future version. Explicitly remove unwanted columns after parsing instead.
  df = pd.read_csv(file, parse_dates=True, keep_date_col=True)


Memory usage of dataframe is 1782.17 MB
Memory usage after optimization is: 443.79 MB
Decreased by 75.1%


C:\Users\malyarchuk\AppData\Local\Temp\ipykernel_24244\147790395.py:42: FutureWarning: The 'keep_date_col' keyword in pd.read_csv is deprecated and will be removed in a future version. Explicitly remove unwanted columns after parsing instead.
  df = pd.read_csv(file, parse_dates=True, keep_date_col=True)


Memory usage of dataframe is 197.38 MB
Memory usage after optimization is: 49.15 MB
Decreased by 75.1%
Run 1/50 started...



c:\Users\malyarchuk\Downloads\nns\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\malyarchuk\Downloads\nns\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\malyarchuk\Downloads\nns\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu


Aggregated Metrics for 1 runs:
test_accuracy: 0.9999
balanced_accuracy: 0.7604
f2: 0.9999
mcc: 0.9985
0_precision: 0.9999
0_recall: 1.0000
0_f1-score: 1.0000
0_support: 194557.0000
1_precision: 0.9991
1_recall: 0.9994
1_f1-score: 0.9992
1_support: 3178.0000
2_precision: 0.9984
2_recall: 1.0000
2_f1-score: 0.9992
2_support: 2496.0000
3_precision: 0.9995
3_recall: 0.9981
3_f1-score: 0.9988
3_support: 2083.0000
4_precision: 1.0000
4_recall: 0.9935
4_f1-score: 0.9967
4_support: 463.0000
5_precision: 1.0000
5_recall: 1.0000
5_f1-score: 1.0000
5_support: 441.0000
6_precision: 0.9950
6_recall: 0.9853
6_f1-score: 0.9901
6_support: 204.0000
7_precision: 1.0000
7_recall: 1.0000
7_f1-score: 1.0000
7_support: 196.0000
8_precision: 1.0000
8_recall: 1.0000
8_f1-score: 1.0000
8_support: 53.0000
9_precision: 1.0000
9_recall: 0.9091
9_f1-score: 0.9524
9_support: 11.0000
10_precision: 0.8333
10_recall: 0.8333
10_f1-score: 0.8333
10_support: 6.0000
11_precision: 1.0000
11_recall: 1.0000
11_f1-score: 1.0

c:\Users\malyarchuk\Downloads\nns\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\malyarchuk\Downloads\nns\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\malyarchuk\Downloads\nns\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu


Aggregated Metrics for 2 runs:
test_accuracy: 0.9999
balanced_accuracy: 0.7721
f2: 0.9999
mcc: 0.9984
0_precision: 0.9999
0_recall: 1.0000
0_f1-score: 0.9999
0_support: 194557.0000
1_precision: 0.9992
1_recall: 0.9994
1_f1-score: 0.9993
1_support: 3178.0000
2_precision: 0.9982
2_recall: 0.9994
2_f1-score: 0.9988
2_support: 2496.0000
3_precision: 0.9998
3_recall: 0.9974
3_f1-score: 0.9986
3_support: 2083.0000
4_precision: 0.9978
4_recall: 0.9935
4_f1-score: 0.9957
4_support: 463.0000
5_precision: 1.0000
5_recall: 1.0000
5_f1-score: 1.0000
5_support: 441.0000
6_precision: 0.9950
6_recall: 0.9828
6_f1-score: 0.9889
6_support: 204.0000
7_precision: 1.0000
7_recall: 1.0000
7_f1-score: 1.0000
7_support: 196.0000
8_precision: 1.0000
8_recall: 1.0000
8_f1-score: 1.0000
8_support: 53.0000
9_precision: 1.0000
9_recall: 0.9091
9_f1-score: 0.9524
9_support: 11.0000
10_precision: 0.9167
10_recall: 0.8333
10_f1-score: 0.8712
10_support: 6.0000
11_precision: 1.0000
11_recall: 1.0000
11_f1-score: 1.0

c:\Users\malyarchuk\Downloads\nns\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\malyarchuk\Downloads\nns\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\malyarchuk\Downloads\nns\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu


Aggregated Metrics for 3 runs:
test_accuracy: 0.9999
balanced_accuracy: 0.7801
f2: 0.9999
mcc: 0.9985
0_precision: 0.9999
0_recall: 1.0000
0_f1-score: 1.0000
0_support: 194557.0000
1_precision: 0.9993
1_recall: 0.9996
1_f1-score: 0.9994
1_support: 3178.0000
2_precision: 0.9981
2_recall: 0.9993
2_f1-score: 0.9987
2_support: 2496.0000
3_precision: 0.9997
3_recall: 0.9976
3_f1-score: 0.9986
3_support: 2083.0000
4_precision: 0.9986
4_recall: 0.9935
4_f1-score: 0.9960
4_support: 463.0000
5_precision: 1.0000
5_recall: 0.9992
5_f1-score: 0.9996
5_support: 441.0000
6_precision: 0.9950
6_recall: 0.9837
6_f1-score: 0.9893
6_support: 204.0000
7_precision: 1.0000
7_recall: 1.0000
7_f1-score: 1.0000
7_support: 196.0000
8_precision: 1.0000
8_recall: 1.0000
8_f1-score: 1.0000
8_support: 53.0000
9_precision: 1.0000
9_recall: 0.9091
9_f1-score: 0.9524
9_support: 11.0000
10_precision: 0.8889
10_recall: 0.8333
10_f1-score: 0.8586
10_support: 6.0000
11_precision: 1.0000
11_recall: 1.0000
11_f1-score: 1.0

In [ ]:
labels_map = {
    'Normal': 0, 'Darknet_Audio-Streaming': 1, 'Darknet_Chat': 2, 'Darknet_File-Transfer': 3, 'Darknet_VOIP': 4,
    'Darknet_Video-Streaming': 5, 'Darknet_Email': 6, 'Darknet_Browsing': 7, 'Darknet_P2P': 8
}

run_model("/kaggle/input/ma-bsmote/darknet_bsmote.csv", "/kaggle/input/ma-datasets/darknet_test.csv", is_string_labels = True, label_mapping = labels_map)

Run 1/50 started...



Aggregated Metrics for 1 runs:
test_accuracy: 0.9996
balanced_accuracy: 0.9874
f2: 0.9996
mcc: 0.9984
0_precision: 0.9999
0_recall: 1.0000
0_f1-score: 0.9999
0_support: 26862.0000
1_precision: 1.0000
1_recall: 0.9996
1_f1-score: 0.9998
1_support: 2657.0000
2_precision: 0.9956
2_recall: 0.9989
2_f1-score: 0.9973
2_support: 908.0000
3_precision: 1.0000
3_recall: 0.9962
3_f1-score: 0.9981
3_support: 522.0000
4_precision: 0.9898
4_recall: 0.9898
4_f1-score: 0.9898
4_support: 293.0000
5_precision: 0.9926
5_recall: 0.9963
5_f1-score: 0.9944
5_support: 269.0000
6_precision: 1.0000
6_recall: 1.0000
6_f1-score: 1.0000
6_support: 116.0000
7_precision: 0.9796
7_recall: 0.9057
7_f1-score: 0.9412
7_support: 53.0000
8_precision: 0.9778
8_recall: 1.0000
8_f1-score: 0.9888
8_support: 44.0000
accuracy: 0.9996
macro avg_precision: 0.9928
macro avg_recall: 0.9874
macro avg_f1-score: 0.9899
macro avg_support: 31724.0000
weighted avg_precision: 0.9996
weighted avg_recall: 0.9996
weighted avg_f1-score: 0.9


Aggregated Metrics for 2 runs:
test_accuracy: 0.9995
balanced_accuracy: 0.9865
f2: 0.9995
mcc: 0.9983
0_precision: 0.9999
0_recall: 1.0000
0_f1-score: 0.9999
0_support: 26862.0000
1_precision: 1.0000
1_recall: 0.9994
1_f1-score: 0.9997
1_support: 2657.0000
2_precision: 0.9962
2_recall: 0.9983
2_f1-score: 0.9972
2_support: 908.0000
3_precision: 1.0000
3_recall: 0.9952
3_f1-score: 0.9976
3_support: 522.0000
4_precision: 0.9915
4_recall: 0.9915
4_f1-score: 0.9915
4_support: 293.0000
5_precision: 0.9926
5_recall: 0.9981
5_f1-score: 0.9954
5_support: 269.0000
6_precision: 0.9915
6_recall: 1.0000
6_f1-score: 0.9957
6_support: 116.0000
7_precision: 0.9694
7_recall: 0.8962
7_f1-score: 0.9314
7_support: 53.0000
8_precision: 0.9778
8_recall: 1.0000
8_f1-score: 0.9888
8_support: 44.0000
accuracy: 0.9995
macro avg_precision: 0.9910
macro avg_recall: 0.9865
macro avg_f1-score: 0.9886
macro avg_support: 31724.0000
weighted avg_precision: 0.9995
weighted avg_recall: 0.9995
weighted avg_f1-score: 0.9


Aggregated Metrics for 3 runs:
test_accuracy: 0.9995
balanced_accuracy: 0.9859
f2: 0.9995
mcc: 0.9983
0_precision: 0.9999
0_recall: 1.0000
0_f1-score: 0.9999
0_support: 26862.0000
1_precision: 0.9999
1_recall: 0.9995
1_f1-score: 0.9997
1_support: 2657.0000
2_precision: 0.9960
2_recall: 0.9978
2_f1-score: 0.9969
2_support: 908.0000
3_precision: 0.9994
3_recall: 0.9949
3_f1-score: 0.9971
3_support: 522.0000
4_precision: 0.9920
4_recall: 0.9920
4_f1-score: 0.9920
4_support: 293.0000
5_precision: 0.9938
5_recall: 0.9988
5_f1-score: 0.9963
5_support: 269.0000
6_precision: 0.9944
6_recall: 0.9971
6_f1-score: 0.9957
6_support: 116.0000
7_precision: 0.9596
7_recall: 0.8931
7_f1-score: 0.9251
7_support: 53.0000
8_precision: 0.9778
8_recall: 1.0000
8_f1-score: 0.9888
8_support: 44.0000
accuracy: 0.9995
macro avg_precision: 0.9903
macro avg_recall: 0.9859
macro avg_f1-score: 0.9879
macro avg_support: 31724.0000
weighted avg_precision: 0.9995
weighted avg_recall: 0.9995
weighted avg_f1-score: 0.9


Aggregated Metrics for 4 runs:
test_accuracy: 0.9995
balanced_accuracy: 0.9856
f2: 0.9995
mcc: 0.9982
0_precision: 0.9999
0_recall: 1.0000
0_f1-score: 0.9999
0_support: 26862.0000
1_precision: 0.9999
1_recall: 0.9994
1_f1-score: 0.9997
1_support: 2657.0000
2_precision: 0.9956
2_recall: 0.9978
2_f1-score: 0.9967
2_support: 908.0000
3_precision: 0.9981
3_recall: 0.9943
3_f1-score: 0.9962
3_support: 522.0000
4_precision: 0.9923
4_recall: 0.9923
4_f1-score: 0.9923
4_support: 293.0000
5_precision: 0.9945
5_recall: 0.9991
5_f1-score: 0.9968
5_support: 269.0000
6_precision: 0.9936
6_recall: 0.9914
6_f1-score: 0.9924
6_support: 116.0000
7_precision: 0.9597
7_recall: 0.8962
7_f1-score: 0.9269
7_support: 53.0000
8_precision: 0.9833
8_recall: 1.0000
8_f1-score: 0.9916
8_support: 44.0000
accuracy: 0.9995
macro avg_precision: 0.9908
macro avg_recall: 0.9856
macro avg_f1-score: 0.9880
macro avg_support: 31724.0000
weighted avg_precision: 0.9995
weighted avg_recall: 0.9995
weighted avg_f1-score: 0.9

...


Aggregated Metrics for 14 runs:
test_accuracy: 0.9995
balanced_accuracy: 0.9850
f2: 0.9995
mcc: 0.9980
0_precision: 0.9999
0_recall: 1.0000
0_f1-score: 0.9999
0_support: 26862.0000
1_precision: 0.9998
1_recall: 0.9992
1_f1-score: 0.9995
1_support: 2657.0000
2_precision: 0.9958
2_recall: 0.9971
2_f1-score: 0.9964
2_support: 908.0000
3_precision: 0.9982
3_recall: 0.9937
3_f1-score: 0.9960
3_support: 522.0000
4_precision: 0.9917
4_recall: 0.9924
4_f1-score: 0.9921
4_support: 293.0000
5_precision: 0.9924
5_recall: 0.9989
5_f1-score: 0.9956
5_support: 269.0000
6_precision: 0.9884
6_recall: 0.9920
6_f1-score: 0.9902
6_support: 116.0000
7_precision: 0.9516
7_recall: 0.8949
7_f1-score: 0.9223
7_support: 53.0000
8_precision: 0.9888
8_recall: 0.9968
8_f1-score: 0.9927
8_support: 44.0000
accuracy: 0.9995
macro avg_precision: 0.9896
macro avg_recall: 0.9850
macro avg_f1-score: 0.9872
macro avg_support: 31724.0000
weighted avg_precision: 0.9995
weighted avg_recall: 0.9995
weighted avg_f1-score: 0.


Aggregated Metrics for 15 runs:
test_accuracy: 0.9995
balanced_accuracy: 0.9853
f2: 0.9995
mcc: 0.9981
0_precision: 0.9999
0_recall: 1.0000
0_f1-score: 0.9999
0_support: 26862.0000
1_precision: 0.9998
1_recall: 0.9993
1_f1-score: 0.9996
1_support: 2657.0000
2_precision: 0.9959
2_recall: 0.9971
2_f1-score: 0.9965
2_support: 908.0000
3_precision: 0.9982
3_recall: 0.9939
3_f1-score: 0.9960
3_support: 522.0000
4_precision: 0.9918
4_recall: 0.9925
4_f1-score: 0.9922
4_support: 293.0000
5_precision: 0.9926
5_recall: 0.9990
5_f1-score: 0.9958
5_support: 269.0000
6_precision: 0.9886
6_recall: 0.9920
6_f1-score: 0.9902
6_support: 116.0000
7_precision: 0.9522
7_recall: 0.8969
7_f1-score: 0.9236
7_support: 53.0000
8_precision: 0.9896
8_recall: 0.9970
8_f1-score: 0.9932
8_support: 44.0000
accuracy: 0.9995
macro avg_precision: 0.9898
macro avg_recall: 0.9853
macro avg_f1-score: 0.9874
macro avg_support: 31724.0000
weighted avg_precision: 0.9995
weighted avg_recall: 0.9995
weighted avg_f1-score: 0.


Aggregated Metrics for 16 runs:
test_accuracy: 0.9995
balanced_accuracy: 0.9854
f2: 0.9995
mcc: 0.9981
0_precision: 0.9999
0_recall: 1.0000
0_f1-score: 0.9999
0_support: 26862.0000
1_precision: 0.9999
1_recall: 0.9993
1_f1-score: 0.9996
1_support: 2657.0000
2_precision: 0.9960
2_recall: 0.9972
2_f1-score: 0.9966
2_support: 908.0000
3_precision: 0.9982
3_recall: 0.9941
3_f1-score: 0.9962
3_support: 522.0000
4_precision: 0.9919
4_recall: 0.9927
4_f1-score: 0.9923
4_support: 293.0000
5_precision: 0.9926
5_recall: 0.9991
5_f1-score: 0.9958
5_support: 269.0000
6_precision: 0.9888
6_recall: 0.9919
6_f1-score: 0.9903
6_support: 116.0000
7_precision: 0.9539
7_recall: 0.8974
7_f1-score: 0.9247
7_support: 53.0000
8_precision: 0.9902
8_recall: 0.9972
8_f1-score: 0.9936
8_support: 44.0000
accuracy: 0.9995
macro avg_precision: 0.9902
macro avg_recall: 0.9854
macro avg_f1-score: 0.9877
macro avg_support: 31724.0000
weighted avg_precision: 0.9995
weighted avg_recall: 0.9995
weighted avg_f1-score: 0.


Aggregated Metrics for 17 runs:
test_accuracy: 0.9995
balanced_accuracy: 0.9855
f2: 0.9995
mcc: 0.9981
0_precision: 0.9999
0_recall: 1.0000
0_f1-score: 0.9999
0_support: 26862.0000
1_precision: 0.9999
1_recall: 0.9992
1_f1-score: 0.9996
1_support: 2657.0000
2_precision: 0.9959
2_recall: 0.9972
2_f1-score: 0.9965
2_support: 908.0000
3_precision: 0.9983
3_recall: 0.9941
3_f1-score: 0.9962
3_support: 522.0000
4_precision: 0.9920
4_recall: 0.9930
4_f1-score: 0.9925
4_support: 293.0000
5_precision: 0.9928
5_recall: 0.9991
5_f1-score: 0.9960
5_support: 269.0000
6_precision: 0.9894
6_recall: 0.9924
6_f1-score: 0.9909
6_support: 116.0000
7_precision: 0.9542
7_recall: 0.8968
7_f1-score: 0.9245
7_support: 53.0000
8_precision: 0.9895
8_recall: 0.9973
8_f1-score: 0.9933
8_support: 44.0000
accuracy: 0.9995
macro avg_precision: 0.9902
macro avg_recall: 0.9855
macro avg_f1-score: 0.9877
macro avg_support: 31724.0000
weighted avg_precision: 0.9995
weighted avg_recall: 0.9995
weighted avg_f1-score: 0.


Aggregated Metrics for 18 runs:
test_accuracy: 0.9995
balanced_accuracy: 0.9855
f2: 0.9995
mcc: 0.9981
0_precision: 0.9999
0_recall: 1.0000
0_f1-score: 0.9999
0_support: 26862.0000
1_precision: 0.9999
1_recall: 0.9992
1_f1-score: 0.9996
1_support: 2657.0000
2_precision: 0.9957
2_recall: 0.9972
2_f1-score: 0.9965
2_support: 908.0000
3_precision: 0.9983
3_recall: 0.9940
3_f1-score: 0.9962
3_support: 522.0000
4_precision: 0.9919
4_recall: 0.9930
4_f1-score: 0.9924
4_support: 293.0000
5_precision: 0.9930
5_recall: 0.9990
5_f1-score: 0.9960
5_support: 269.0000
6_precision: 0.9900
6_recall: 0.9923
6_f1-score: 0.9911
6_support: 116.0000
7_precision: 0.9535
7_recall: 0.8973
7_f1-score: 0.9244
7_support: 53.0000
8_precision: 0.9901
8_recall: 0.9975
8_f1-score: 0.9937
8_support: 44.0000
accuracy: 0.9995
macro avg_precision: 0.9902
macro avg_recall: 0.9855
macro avg_f1-score: 0.9878
macro avg_support: 31724.0000
weighted avg_precision: 0.9995
weighted avg_recall: 0.9995
weighted avg_f1-score: 0.